<a href="https://colab.research.google.com/github/dream80/TonyColab/blob/master/NAIFU_NovelAILeaks_CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

基于 4chan 魔改版 NovelAILeaks (NaiFu) 制作。 [来源](https://boards.4channel.org/g/thread/89095460#p89097704)



In [ ]:
#@title ### 0. 检查分配到的GPU配置
!nvidia-smi

In [ ]:
#@title ### 1. 下載NaiFu源码和模型 
#@markdown 下载会需要一些时间，进度会显示在下方

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar 
  
!echo "解压中..."
!tar xf naifu.tar && rm naifu.tar
!echo "搞定"

In [ ]:
#@title ### 2. 安装依赖
#@markdown 耐心等待安装完成

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

In [ ]:
#@title ### 3. 启动服务器
#@markdown 启动成功后，访问以 `trycloudflare.com` / `bore.pub` 结尾的网址即可。

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

In [ ]:
#@title ### 4.（可选）使用 7G 的 `animefull-latest` 模型启动服务。
#@markdown 默认使用的是4G大小的animefull-final-pruned模型。如果想使用 7G 的 animefull-latest 模型。就点击这里。

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969